In [13]:

import cv2 as cv
import numpy as np 
import tensorflow as tf

In [4]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                rescale = 1/255.0,
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.20,
                                                                horizontal_flip=True,
                                                                vertical_flip=True
                                                                )

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                                rescale = 1/255.0,
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.20,
                                                                horizontal_flip=True,
                                                                vertical_flip=True
                                                                )

train_generator = train_datagen.flow_from_directory(
                                                    directory='data/train',
                                                    target_size=(224, 224),
                                                    batch_size=12,
                                                    class_mode='binary',
                                                    shuffle=True,
                                                    seed=42
                                                    )

valid_generator = valid_datagen.flow_from_directory(
                                                    directory='data/validation',
                                                    target_size=(224, 224),
                                                    batch_size=12,
                                                    class_mode='binary',
                                                    shuffle=True,
                                                    seed=42
                                                    )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [10]:
# sequential API
model = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(64, (3,3), input_shape=(224,224,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
                            tf.keras.layers.MaxPool2D(2,2),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(512, activation = 'relu'),
                            tf.keras.layers.Dense(256, activation = 'relu'),
                            tf.keras.layers.Dense(1, activation='sigmoid')    
                        ])
model.summary()
# functional API

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 111, 111, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 54, 54, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 52, 52, 128)       147584    
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 26, 26, 128)      

In [12]:
model.compile(
            loss = 'binary_crossentropy',
            optimizer = 'adam',
            metrics=['accuracy']
            )

model.fit(
        train_generator,
        validation_data=valid_generator,
        epochs=20
        )

Epoch 1/20
86/86 [==============================] - 41s 470ms/step - loss: 0.7101 - accuracy: 0.5229 - val_loss: 0.6461 - val_accuracy: 0.6289
Epoch 2/20
86/86 [==============================] - 41s 478ms/step - loss: 0.5208 - accuracy: 0.7595 - val_loss: 0.8535 - val_accuracy: 0.7695
Epoch 3/20
86/86 [==============================] - 40s 460ms/step - loss: 0.3928 - accuracy: 0.8510 - val_loss: 0.8240 - val_accuracy: 0.7422
Epoch 4/20
86/86 [==============================] - 39s 450ms/step - loss: 0.2925 - accuracy: 0.8802 - val_loss: 1.9620 - val_accuracy: 0.7070
Epoch 5/20
86/86 [==============================] - 40s 465ms/step - loss: 0.2294 - accuracy: 0.9192 - val_loss: 0.8163 - val_accuracy: 0.8320
Epoch 6/20
86/86 [==============================] - 41s 470ms/step - loss: 0.2058 - accuracy: 0.9270 - val_loss: 0.6082 - val_accuracy: 0.8672
Epoch 7/20
86/86 [==============================] - 39s 449ms/step - loss: 0.2722 - accuracy: 0.8997 - val_loss: 1.0258 - val_accuracy: 0.7734

In [16]:
train_generator.class_indices

{'horses': 0, 'humans': 1}

In [19]:
def inference(image_path):
    img = cv.imread(image_path)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (224,224))
    img = img / 255.0
    img = np.array([img])
    P = model.predict(img).squeeze()
    return 'HUMAN' if P > 0.5 else 'HORSE'
    
inference('data/train/horses/horse01-9.png')

1/1 [==============================] - 0s 33ms/step


'HORSE'